## Imagenet Validation Techniques Compared - Square VS Rectangle VS TTA

Here we compare the different validation techniques on a pretrained resnet50 model

In [1]:
%reload_ext autoreload
%autoreload 2

In [2]:
from validation_utils import sort_ar, chunks, map_idx2ar, ValDataset, SequentialIndexSampler, RectangularCropTfm, validate

import sys, os, shutil, time, warnings
from pathlib import Path
import numpy as np
import matplotlib.pyplot as plt
import urllib.request
import pandas as pd

import torch
import torch.nn as nn
import torch.backends.cudnn as cudnn
import torchvision.datasets as datasets
import torchvision.transforms as transforms
from torch.utils.data.sampler import Sampler

Download Tiny ImageNet from https://tiny-imagenet.herokuapp.com/  
`wget -qO- -O tmp.zip http://cs231n.stanford.edu/tiny-imagenet-200.zip && unzip tmp.zip && rm tmp.zip`

In [3]:
imagenet_path = '/data/rliaw/explainability/' # Update the path to point to tiny-imagenet-200

In [4]:
cudnn.benchmark = True

data = Path(imagenet_path + 'tiny-imagenet-200/train')
workers = 7
valdir = imagenet_path + 'tiny-imagenet-200/val/valdir'
batch_size = 64
fp16 = False

#### (Added On) Build Correct Validation Directory File Structure

In [5]:
_path = os.path.join(imagenet_path, 'tiny-imagenet-200/val/val_annotations.txt')
with open(_path) as f:
    anns = [line.rstrip('\n') for line in f]

In [6]:
from shutil import copyfile

val_path = Path(valdir)
if not os.path.exists(val_path):
    os.mkdir(val_path)

for ann in anns:
    img_name, class_id, x0, y0, x1, y1 = ann.split()
    img_path = os.path.join(imagenet_path, 'tiny-imagenet-200/val/images/{}'.format(img_name))
    
    new_path = os.path.join(imagenet_path, 'tiny-imagenet-200/val/valdir/{}'.format(class_id))
    if not os.path.exists(new_path):
        os.mkdir(new_path)
    
    copyfile(img_path, os.path.join(new_path, img_name))

### Step 1: Create Image to Aspect ratio mapping

In [7]:
idx_ar_sorted = sort_ar(data, valdir)

### Step 2: Get untrained resnet model

In [8]:
import resnet
model = resnet.resnet50(pretrained=False)
model = model.cuda()
criterion = nn.CrossEntropyLoss().cuda()
if fp16: model = model.half()

In [9]:
def train_model(model, criterion, optimizer, scheduler, save_path, num_epochs=25, start_epoch=0):
    since = time.time()

    best_model_wts = copy.deepcopy(model.state_dict())
    best_acc = 0.0

    for epoch in range(start_epoch, start_epoch + num_epochs):
        print('Epoch {}/{}'.format(epoch, start_epoch + num_epochs - 1))
        print('-' * 10)

        # Each epoch has a training and validation phase
        for phase in ['train', 'val']:
            if phase == 'train':
                scheduler.step()
                model.train()  # Set model to training mode
            else:
                model.eval()   # Set model to evaluate mode

            running_loss = 0.0
            running_corrects = 0

            # Iterate over data.
            for inputs, labels in dataloaders[phase]:
                inputs = inputs.to(device)
                labels = labels.to(device)
                
                # zero the parameter gradients
                optimizer.zero_grad()

                # forward
                # track history if only in train
                with torch.set_grad_enabled(phase == 'train'):
                    outputs = model(inputs)
                    _, preds = torch.max(outputs, 1)
                    loss = criterion(outputs, labels)

                    # backward + optimize only if in training phase
                    if phase == 'train':
                        loss.backward()
                        optimizer.step()

                # statistics
                running_loss += loss.item() * inputs.size(0)
                running_corrects += torch.sum(preds == labels.data)

            epoch_loss = running_loss / dataset_sizes[phase]
            epoch_acc = running_corrects.double() / dataset_sizes[phase]

            print('{} Loss: {:.4f} Acc: {:.4f}'.format(
                phase, epoch_loss, epoch_acc))

            # deep copy the model
            if phase == 'val' and epoch_acc > best_acc:
                best_acc = epoch_acc
                best_model_wts = copy.deepcopy(model.state_dict())
                
            torch.save(model.state_dict(), "{}/{}.pt".format(save_path, epoch))

        print()

    time_elapsed = time.time() - since
    print('Training complete in {:.0f}m {:.0f}s'.format(
        time_elapsed // 60, time_elapsed % 60))
    print('Best val Acc: {:4f}'.format(best_acc))

    # load best model weights
    model.load_state_dict(best_model_wts)
    return model

In [10]:
import torchvision

dataset_sizes = {'train': 100000,
                 'val':    10000}

trn_tfms = [transforms.ToTensor()]
trn_img_folder = torchvision.datasets.ImageFolder(data, transforms.Compose(trn_tfms))
val_img_folder = torchvision.datasets.ImageFolder(valdir, transforms.Compose(trn_tfms))

dataloaders = {'train': torch.utils.data.DataLoader(trn_img_folder, shuffle=True),
               'val':   torch.utils.data.DataLoader(val_img_folder, shuffle=True)}

In [ ]:
model_ft = model
num_ftrs = model_ft.fc.in_features
model_ft.fc = nn.Linear(num_ftrs, 200)

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
model_ft = model_ft.to(device)

import copy
from torch import optim
# Observe that all parameters are being optimized
optimizer_ft = optim.SGD(model_ft.parameters(), lr=0.001, momentum=0.9)

# Decay LR by a factor of 0.1 every 7 epochs
exp_lr_scheduler = optim.lr_scheduler.StepLR(optimizer_ft, step_size=7, gamma=0.1)

model_ft = train_model(model_ft, criterion, optimizer_ft, exp_lr_scheduler,
                       num_epochs=2, save_path="/data/rliaw/explainability/temp")

In [ ]:
model_ft = resnet.resnet50(pretrained=False).cuda()

num_ftrs = model_ft.fc.in_features
model_ft.fc = nn.Linear(num_ftrs, 200)

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
model_ft = model_ft.to(device)

import copy
from torch import optim
# Observe that all parameters are being optimized
optimizer_ft = optim.SGD(model_ft.parameters(), lr=0.001, momentum=0.9)

# Decay LR by a factor of 0.1 every 7 epochs
exp_lr_scheduler = optim.lr_scheduler.StepLR(optimizer_ft, step_size=7, gamma=0.1)

# Load model state from given epoch num
epoch_num = 1
model_ft.load_state_dict(torch.load("/data/rliaw/explainability/temp/{}.pt".format(epoch_num)))

model_ft = train_model(model_ft, criterion, optimizer_ft, exp_lr_scheduler, num_epochs=5, start_epoch=epoch_num+1,
                       save_path="/data/rliaw/explainability/temp")

In [ ]:
model_ft = resnet.resnet50(pretrained=False).cuda()

num_ftrs = model_ft.fc.in_features
model_ft.fc = nn.Linear(num_ftrs, 200)

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
model_ft = model_ft.to(device)

import copy
from torch import optim
# Observe that all parameters are being optimized
optimizer_ft = optim.SGD(model_ft.parameters(), lr=0.001, momentum=0.9)

# Decay LR by a factor of 0.1 every 7 epochs
exp_lr_scheduler = optim.lr_scheduler.StepLR(optimizer_ft, step_size=7, gamma=0.1)

# Load model state from given epoch num
epoch_num = 4
model_ft.load_state_dict(torch.load("/data/rliaw/explainability/temp/{}.pt".format(epoch_num)))

model_ft = train_model(model_ft, criterion, optimizer_ft, exp_lr_scheduler, num_epochs=4, start_epoch=epoch_num+1,
                       save_path="/data/rliaw/explainability/temp")

Epoch 5/8
----------
train Loss: 3.4801 Acc: 0.2202
val Loss: 13.8784 Acc: 0.0226

Epoch 6/8
----------
train Loss: 3.1876 Acc: 0.2687
val Loss: 8.8846 Acc: 0.0264

Epoch 7/8
----------


In [ ]:
model_ft = resnet.resnet50(pretrained=False).cuda()

num_ftrs = model_ft.fc.in_features
model_ft.fc = nn.Linear(num_ftrs, 200)

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
model_ft = model_ft.to(device)

import copy
from torch import optim
# Observe that all parameters are being optimized
optimizer_ft = optim.SGD(model_ft.parameters(), lr=0.001, momentum=0.9)

# Decay LR by a factor of 0.1 every 7 epochs
exp_lr_scheduler = optim.lr_scheduler.StepLR(optimizer_ft, step_size=7, gamma=0.1)

# Load model state from given epoch num
epoch_num = 7
model_ft.load_state_dict(torch.load("/data/rliaw/explainability/temp/{}.pt".format(epoch_num)))

model_ft = train_model(model_ft, criterion, optimizer_ft, exp_lr_scheduler, num_epochs=7, start_epoch=epoch_num+1,
                       save_path="/data/rliaw/explainability/temp")

Epoch 8/14
----------
train Loss: 2.6158 Acc: 0.3714
val Loss: 14.3492 Acc: 0.0220

Epoch 9/14
----------
train Loss: 2.3324 Acc: 0.4264
val Loss: 18.0368 Acc: 0.0189

Epoch 10/14
----------


In [ ]:
model_ft = resnet.resnet50(pretrained=False).cuda()

num_ftrs = model_ft.fc.in_features
model_ft.fc = nn.Linear(num_ftrs, 200)

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
model_ft = model_ft.to(device)

import copy
from torch import optim
# Observe that all parameters are being optimized
optimizer_ft = optim.SGD(model_ft.parameters(), lr=0.001, momentum=0.9)

# Decay LR by a factor of 0.1 every 7 epochs
exp_lr_scheduler = optim.lr_scheduler.StepLR(optimizer_ft, step_size=7, gamma=0.1)

# Load model state from given epoch num
epoch_num = 9
model_ft.load_state_dict(torch.load("/data/rliaw/explainability/temp/{}.pt".format(epoch_num)))

model_ft = train_model(model_ft, criterion, optimizer_ft, exp_lr_scheduler, num_epochs=5, start_epoch=epoch_num+1,
                       save_path="/data/rliaw/explainability/temp")

Epoch 10/14
----------
train Loss: 2.0186 Acc: 0.4898
val Loss: 15.3556 Acc: 0.0251

Epoch 11/14
----------
train Loss: 1.7182 Acc: 0.5544
val Loss: 20.9767 Acc: 0.0211

Epoch 12/14
----------
train Loss: 1.3902 Acc: 0.6292
val Loss: 20.9082 Acc: 0.0214

Epoch 13/14
----------


In [ ]:
model_ft = resnet.resnet50(pretrained=False).cuda()

num_ftrs = model_ft.fc.in_features
model_ft.fc = nn.Linear(num_ftrs, 200)

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
model_ft = model_ft.to(device)

import copy
from torch import optim
# Observe that all parameters are being optimized
optimizer_ft = optim.SGD(model_ft.parameters(), lr=0.001, momentum=0.9)

# Decay LR by a factor of 0.1 every 7 epochs
exp_lr_scheduler = optim.lr_scheduler.StepLR(optimizer_ft, step_size=7, gamma=0.1)

# Load model state from given epoch num
epoch_num = 12
model_ft.load_state_dict(torch.load("/data/rliaw/explainability/temp/{}.pt".format(epoch_num)))

model_ft = train_model(model_ft, criterion, optimizer_ft, exp_lr_scheduler, num_epochs=3, start_epoch=epoch_num+1,
                       save_path="/data/rliaw/explainability/temp")

Epoch 13/15
----------


#### Global dataset settings

In [9]:
val_bs = 64
target_size = 288

idx_sorted, _ = zip(*idx_ar_sorted)
idx2ar, ar_means = map_idx2ar(idx_ar_sorted, val_bs)
val_sampler_ar = SequentialIndexSampler(idx_sorted)

normalize = transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
tensor_tfm = [transforms.ToTensor(), normalize]

## Compare different Validations

### Test Square Validation Technique

Resize Image 1.14x -> Crop to target size (288)

In [10]:
val_tfms = [transforms.Resize(int(target_size*1.14)), transforms.CenterCrop(target_size)] + tensor_tfm
val_dataset = datasets.ImageFolder(valdir,  transforms.Compose(val_tfms))

val_loader = torch.utils.data.DataLoader(
    val_dataset, batch_size=val_bs, shuffle=False,
    num_workers=workers, pin_memory=True, sampler=val_sampler_ar)

orig_prec5 = validate(val_loader, model, criterion)

Test: [100/157]	Time 0.142 (0.170)	Loss 34.9903 (33.6966)	Prec@1 0.000 (0.031)	Prec@5 0.000 (0.234)
Test: [157/157]	Time 0.892 (0.165)	Loss 28.6202 (33.0846)	Prec@1 0.000 (0.040)	Prec@5 0.000 (0.220)
Total Time:0.007190988888888889	 Top 5 Accuracy: 0.220

 * Prec@1 0.040 Prec@5 0.220


### Test Fast.Ai Rectangular Validation

Perform validation with rectangular images!

In [11]:
val_ar_tfms = [transforms.Resize(int(target_size*1.14)), RectangularCropTfm(idx2ar, target_size)]
val_dataset_rect = ValDataset(valdir, val_ar_tfms+tensor_tfm)
val_loader = torch.utils.data.DataLoader(
    val_dataset_rect, batch_size=val_bs, shuffle=False,
    num_workers=workers, pin_memory=True, sampler=val_sampler_ar)

rect_prec5 = validate(val_loader, model, criterion)

Test: [100/157]	Time 0.143 (0.152)	Loss 34.9903 (33.6966)	Prec@1 0.000 (0.031)	Prec@5 0.000 (0.234)
Test: [157/157]	Time 0.040 (0.148)	Loss 28.6202 (33.0846)	Prec@1 0.000 (0.040)	Prec@5 0.000 (0.220)
Total Time:0.006467484999999999	 Top 5 Accuracy: 0.220

 * Prec@1 0.040 Prec@5 0.220


## Comparison Square VS Rectangles

In [12]:
def batch_mean(array, size=10): return [np.array(c).mean() for c in chunks(array, 100)]
batch_means = batch_mean(ar_means)
rect_prec5_mean = batch_mean(rect_prec5)
orig_prec5_mean = batch_mean(orig_prec5)

In [13]:
d = {'OriginalValidation': orig_prec5_mean, 
     'RectangularValidation': rect_prec5_mean, 
     'AR Mean': batch_means,
     'Difference': np.array(rect_prec5_mean)-np.array(orig_prec5_mean)}
df = pd.DataFrame(data=d); df

,OriginalValidation,RectangularValidation,AR Mean,Difference
0,0.234375,0.234375,1.0,0.0
1,0.191886,0.191886,1.0,0.0


### Validate with TTA (Test Time Augmentation)

Take 4 random crops + original validation image and averages the predictions together

In [14]:
min_scale = 0.5
trn_tfms = [
        transforms.RandomResizedCrop(target_size, scale=(min_scale, 1.0)),
        transforms.RandomHorizontalFlip(),
    ] + tensor_tfm
aug_dataset = datasets.ImageFolder(valdir, transforms.Compose(trn_tfms))

val_tfms = [transforms.Resize(int(target_size*1.14)), transforms.CenterCrop(target_size)] + tensor_tfm
val_dataset = datasets.ImageFolder(valdir,  transforms.Compose(val_tfms))

aug_loader = torch.utils.data.DataLoader(
    aug_dataset, batch_size=val_bs, shuffle=False,
    num_workers=workers, pin_memory=True, sampler=val_sampler_ar)

val_loader = torch.utils.data.DataLoader(
    val_dataset, batch_size=val_bs, shuffle=False,
    num_workers=workers, pin_memory=True, sampler=val_sampler_ar)


In [15]:
tta_prec5 = validate(val_loader, model, criterion, aug_loader=aug_loader, num_augmentations=4)

Test: [100/157]	Time 0.721 (0.756)	Loss 34.3519 (32.7869)	Prec@1 0.000 (0.016)	Prec@5 0.000 (0.250)
Test: [157/157]	Time 0.198 (0.741)	Loss 29.1596 (32.1386)	Prec@1 0.000 (0.030)	Prec@5 0.000 (0.200)
Total Time:0.032309113888888887	 Top 5 Accuracy: 0.200

 * Prec@1 0.030 Prec@5 0.200


### Validate with TTA and Rectangles

Take 4 random crops + recangular validation image and averages the predictions together

In [16]:
min_scale = 0.5
trn_tfms = [
        transforms.RandomResizedCrop(target_size, scale=(min_scale, 1.0)),
        transforms.RandomHorizontalFlip(),
    ] + tensor_tfm
aug_dataset = datasets.ImageFolder(valdir, transforms.Compose(trn_tfms))

aug_loader = torch.utils.data.DataLoader(
    aug_dataset, batch_size=val_bs, shuffle=False,
    num_workers=workers, pin_memory=True, sampler=val_sampler_ar)

val_ar_tfms = [transforms.Resize(int(target_size*1.14)), RectangularCropTfm(idx2ar, target_size)]
val_dataset_rect = ValDataset(valdir, val_ar_tfms+tensor_tfm)
val_loader = torch.utils.data.DataLoader(
    val_dataset_rect, batch_size=val_bs, shuffle=False,
    num_workers=workers, pin_memory=True, sampler=val_sampler_ar)

tta_rect_prec5 = validate(val_loader, model, criterion, aug_loader=aug_loader, num_augmentations=4)

Test: [100/157]	Time 0.724 (0.741)	Loss 34.6764 (32.7518)	Prec@1 0.000 (0.016)	Prec@5 0.000 (0.250)
Test: [157/157]	Time 0.199 (0.731)	Loss 28.8469 (32.1384)	Prec@1 0.000 (0.030)	Prec@5 0.000 (0.230)
Total Time:0.03188365861111111	 Top 5 Accuracy: 0.230

 * Prec@1 0.030 Prec@5 0.230


## Comparing all the Techniques

In [17]:
def batch_mean(array, size=10): return [np.array(c).mean() for c in chunks(array, 100)]
batch_means = batch_mean(ar_means)
rect_prec5_mean = batch_mean(rect_prec5)
orig_prec5_mean = batch_mean(orig_prec5)
tta_prec5_mean = batch_mean(tta_prec5)
tta_rect_prec5_mean = batch_mean(tta_rect_prec5)

In [18]:
d = {'Original Validation': orig_prec5_mean, 
     'Rectangular Validation': rect_prec5_mean, 
     'TTA Validation': tta_prec5_mean, 
     'TTA + Rectangular Validation': tta_rect_prec5_mean, 
     'AR Mean': batch_means}
df = pd.DataFrame(data=d); df

,Original Validation,Rectangular Validation,TTA Validation,TTA + Rectangular Validation,AR Mean
0,0.234375,0.234375,0.250000,0.250000,1.0
1,0.191886,0.191886,0.109649,0.191886,1.0
